# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [149]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [150]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

['Frog', 'Stand', 'Right', 'Back', 'Sit', 'HeadBack', 'Belly', 'Crouch', 'StandInit', 'Knee', 'Left']


In [151]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE 
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [classes[i]] * len(data)
    return data, target

In [152]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for i in range(len(classes)):
    data, target = load_pose_data(i)
    all_data.extend(data)
    all_target.extend(target)
    print target[0], len(target)

print 'total number of data', len(all_data)

Frog 10
Stand 11
Right 20
Back 12
Sit 26
HeadBack 10
Belly 19
Crouch 36
StandInit 52
Knee 10
Left 20
total number of data 226


In [153]:
# shuffule data
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * 0.7)
training_data = permutation[:n_training_data]

## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [154]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [155]:
# YOUR CODE HERE
training_d = []
training_t = []

for i in range(len(training_data)):
    training_d.append(all_data[training_data[i]])
    training_t.append(all_target[training_data[i]])

clf.fit(training_d, training_t)

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### predicting

In [166]:
clf.predict(all_data[1:])[0], all_target[-1:]

('Frog', ['Left'])

In [157]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [158]:
expected = []
predicted = []
# YOUR CODE HERE
expected.extend(training_t)
predicted.extend(clf.predict(training_d))

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

       Back       1.00      1.00      1.00        11
      Belly       1.00      1.00      1.00        12
     Crouch       0.97      1.00      0.98        28
       Frog       1.00      1.00      1.00         6
   HeadBack       1.00      1.00      1.00         6
       Knee       1.00      1.00      1.00         8
       Left       1.00      1.00      1.00        13
      Right       1.00      0.92      0.96        13
        Sit       1.00      1.00      1.00        20
      Stand       1.00      1.00      1.00         7
  StandInit       1.00      1.00      1.00        34

avg / total       0.99      0.99      0.99       158


Confusion matrix:
[[11  0  0  0  0  0  0  0  0  0  0]
 [ 0 12  0  0  0  0  0  0  0  0  0]
 [ 0  0 28  0  0  0  0  0  0  0  0]
 [ 0  0  0  6  0  0  0  0  0  0  0]
 [ 0  0  0  0  6  0  0  0  0  0  0]
 [ 0  0  0  0  0  8  0  0  0  0  0]
 [ 0  0  0  0  0  0 13  0  0  0  0]
 [ 0  0  1  0 


## 4. Evaluate on the test data

In [159]:
expected = []
predicted = []
# YOUR CODE HERE

expected.extend(all_target)
predicted.extend(clf.predict(all_data))

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

       Back       1.00      1.00      1.00        12
      Belly       1.00      1.00      1.00        19
     Crouch       0.95      1.00      0.97        36
       Frog       1.00      1.00      1.00        10
   HeadBack       1.00      1.00      1.00        10
       Knee       1.00      1.00      1.00        10
       Left       1.00      1.00      1.00        20
      Right       1.00      0.90      0.95        20
        Sit       1.00      1.00      1.00        26
      Stand       1.00      1.00      1.00        11
  StandInit       1.00      1.00      1.00        52

avg / total       0.99      0.99      0.99       226


Confusion matrix:
[[12  0  0  0  0  0  0  0  0  0  0]
 [ 0 19  0  0  0  0  0  0  0  0  0]
 [ 0  0 36  0  0  0  0  0  0  0  0]
 [ 0  0  0 10  0  0  0  0  0  0  0]
 [ 0  0  0  0 10  0  0  0  0  0  0]
 [ 0  0  0  0  0 10  0  0  0  0  0]
 [ 0  0  0  0  0  0 20  0  0  0  0]
 [ 0  0  2  0 

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [114]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [115]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1:]), all_target[-1:]

(array(['Left'], 
       dtype='|S9'), ['Left'])